In [ ]:
!pip install transformers torchaudio jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.3 MB/s eta 0:00:00


In [ ]:
import multiprocessing as mp

try:
   mp.set_start_method('spawn', force=True)
   print("spawned")
except RuntimeError:
   pass

spawned


In [ ]:
import torch

torch.multiprocessing.set_start_method('spawn', force=True)

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import AutoModelForCTC, AutoProcessor
import torchaudio
import jiwer
import pandas as pd

# Load the pre-trained model and processor
model_name = "facebook/wav2vec2-base-960h"
#model_name = "patrickvonplaten/wav2vec2-base-960h-4-gram"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name).to("cuda")
#


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [ ]:
# Set the LibriSpeech dataset configuration
librispeech_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=True)

100%|██████████| 331M/331M [00:34<00:00, 9.92MB/s]


In [ ]:
# Process each example in the dataset
def evaluation(librispeech_dataset):
  predictions = []
  ground_truths = []
  ids = []
  for i in range(len(librispeech_dataset)):
      waveform, sample_rate, utterance, speaker_id, sentence_id, number = librispeech_dataset[i]
      # Preprocess the audio file
      input_values = processor(waveform.numpy(), sampling_rate = sample_rate,
                               return_tensors="pt").input_values.to("cuda")

      # Perform inference
      with torch.no_grad():
          logits = model(input_values).logits

      # Decode the logits to obtain the transcriptions
      predicted_ids = torch.argmax(logits, dim=-1)
      transcription = processor.batch_decode(predicted_ids.cpu().numpy())[0]
      predictions.append(transcription)
      ground_truths.append(utterance)
      prefix = "0" * (4-len(str(number)))
      id = f"{speaker_id}-{sentence_id}-{prefix}{number}"
      ids.append(id)
  return predictions, ground_truths,ids

In [ ]:
predictions, ground_truths,ids = evaluation(librispeech_dataset)

In [ ]:
jiwer.wer(ground_truths, predictions)

0.0338557516737675

In [ ]:
librispeech_other_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-other", download=True)

100%|██████████| 314M/314M [00:29<00:00, 11.2MB/s]


In [ ]:
predictions_other, ground_truths_other,ids_other = evaluation(librispeech_other_dataset)

In [ ]:
jiwer.wer(ground_truths_other, predictions_other)

0.0856848098122003

In [ ]:
wer_clean = []
for i,g, p in zip(ids,ground_truths, predictions):
  wer_clean.append((i,jiwer.wer(g, p),g,p))

In [ ]:
wer_clean_results = sorted(wer_clean,key=lambda tup: tup[1], reverse=True)
d = pd.DataFrame(wer_clean_results, columns = ['id', 'wer', 'ground_truth', 'prediction'])
d.to_csv('wav2vec_clean.csv', index=False)

In [ ]:
wer_other = []
for i,g, p in zip(ids_other, ground_truths_other, predictions_other):
  wer_other.append((i,jiwer.wer(g, p),g,p))

In [ ]:
wer_other_results = sorted(wer_other,key=lambda tup: tup[1], reverse=True)
d1 = pd.DataFrame(wer_other_results, columns = ['id', 'wer', 'ground_truth', 'prediction'])
d1.to_csv('wav2vec_other.csv', index=False)